This is the only notebook to run denoising techniques on I am going to do all the work here, once the work is done I will take it to scripts


In [88]:
import scanpy as sc
import argparse
import os
import sys
import time
import tracemalloc  # For memory profiling
from scipy import sparse
import scanpy.external as sce
import numpy as np
import doubletdetection
import warnings
import logging
warnings.filterwarnings("ignore", category=UserWarning, module="scanpy")
# Set up logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
log = logging.getLogger("qualityControl")
log.setLevel(logging.INFO)

In [90]:
context_path ="/home/sah2p/ondemand/singlecell_data/Maize/"
dataset = "SRP145013"
filename = dataset+"_raw.h5ad"
adata_obj = sc.read_h5ad(context_path+filename)


# Split data by sample to perform QC

In [91]:
adata_by_sample = [adata_obj[adata_obj.obs['Orig.ident'] == sample].copy() for sample in adata_obj.obs['Orig.ident'].unique()]

for i, adata_raw in enumerate(adata_by_sample):
    
    adata_raw.var["mt"] = adata_raw.var_names.str.startswith("ZeamMt")
    adata_raw.var["pt"] = adata_raw.var_names.str.startswith("ZemaCt")
        
    sc.pp.calculate_qc_metrics(adata_raw, qc_vars=["mt", "pt"], inplace=True, log1p=True)

    # Visualize distributions before filtering
    sc.pl.violin(
        adata_raw,
        ["n_genes_by_counts", "total_counts", "pct_counts_mt","pct_counts_pt"],
        jitter=0.4,
        multi_panel=True,
        save="_"+str(i)+"_02_before_filtering_qc.png"
    )
    sc.pl.scatter(adata_raw, "total_counts", "n_genes_by_counts", color="pct_counts_mt", save=dataset+"_"+str(i)+"_02_total_counts_vs_n_genes(mt).png")
    sc.pl.scatter(adata_raw, "total_counts", "n_genes_by_counts", color="pct_counts_pt", save=dataset+"_"+str(i)+"_02_total_counts_vs_n_genes(pt).png")
            


    # Step 6: Apply IQR-based filtering
    Q1 = np.percentile(adata_raw.obs["n_genes_by_counts"], 25)  # First quartile
    Q3 = np.percentile(adata_raw.obs["n_genes_by_counts"], 75)  # Third quartile
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    adata_raw = adata_raw[
                    (adata_raw.obs['pct_counts_mt'] <= 10) &
                    (adata_raw.obs['pct_counts_pt'] <= 5) &
                    (adata_raw.obs['n_genes_by_counts'] > lower_bound) &
                    (adata_raw.obs['n_genes_by_counts'] < upper_bound) &
                    (adata_raw.obs['total_counts'] > 200) &
                    (adata_raw.obs['total_counts'] < 10000)]
        # Visualize distributions before filtering
    sc.pl.violin(
        adata_raw,
        ["n_genes_by_counts", "total_counts", "pct_counts_mt","pct_counts_pt"],
        jitter=0.4,
        multi_panel=True,
        save=dataset+"_"+str(i)+"_02_after_filtering_qc.png"
    )
    log.info(f"Remaining cells after filtering: {adata_raw.n_obs}")
        
    # Step 8: Remove empty droplets and doublets
    clf = doubletdetection.BoostClassifier(
    n_iters=10, 
    clustering_algorithm="louvain", 
    standard_scaling=True,
    pseudocount=0.1,
    n_jobs=-1)
    
    doublets = clf.fit(adata_raw.X).predict(p_thresh=1e-16, voter_thresh=0.5)
    doublet_score = clf.doublet_score()
    adata_raw.obs["doublet"] = doublets
    adata_raw.obs["doublet_score"] = doublet_score
    print(adata_raw.obs["doublet"].value_counts())
    
    sc.tl.pca(adata_raw)  # Ensure PCA is computed
    sc.pp.neighbors(adata_raw)  # Compute neighbors graph
    sc.tl.umap(adata_raw)
    sc.pl.umap(adata_raw, color=["doublet", "doublet_score"],save=dataset+"_"+str(i)+"_doublet_umap.png")
    adata_raw = adata_raw[adata_raw.obs['doublet'] == 0, :]
    log.info(f"Remaining cells after doubletRemoval: {adata_raw.n_obs}")
    adata_by_sample[i] = adata_raw
        


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/

/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_utils.py:715: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  color = color[sort]


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_utils.py:715: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  color = color[sort]


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/

  0%|          | 0/10 [00:00<?, ?it/s]

...

/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/

  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_2425043/2076859017.py:58: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_raw.obs["doublet"] = doublets


doublet
0.0    1838
Name: count, dtype: int64


2025-02-21 12:16:20,582 - INFO - Remaining cells after doubletRemoval: 1838


In [27]:
combined_adata = adata_by_sample[0].concatenate(adata_by_sample[1:])
combined_adata.write(context_path+dataset+"_qc_filtered.h5ad")


/tmp/ipykernel_2425043/3772738881.py:1: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  combined_adata = adata_by_sample[0].concatenate(adata_by_sample[1:])


# Preprocessing Data

In [100]:
adata = sc.read_h5ad(context_path+dataset+"_qc_filtered.h5ad")
batch_key = "Orig.ident"
label_key = "Celltype"

In [101]:
# Batch Correction
adata.obs[batch_key].value_counts()


Orig.ident
SRX12403797    10157
SRX12403794     4571
SRX12403795     2782
SRX12403796     2593
SRX12403793     2447
SRX12403792     2364
SRX10657011     1838
SRX10657013      997
SRX10657012      585
Name: count, dtype: int64

In [102]:
#Checking for batch effect
sc.pp.normalize_per_cell(adata)
# sc.pp.log1p(adata)
sc.pp.sqrt(adata) # Because we are applying magic here, and magic is based on square root transformation
adata.layers["sqrtcounts"] = adata.X.copy()

sc.pp.highly_variable_genes(adata)
sc.tl.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)



/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:220: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']


In [103]:
# adata.uns[batch_key + "_colors"] = [
#     "#1b9e77",
#     "#d95f02",
#     "#7570b3",
# ]  # Set custom colours for batches
sc.pl.umap(adata, color=[label_key, batch_key], wspace=1, save="_batch_effect_firstcheck.png")

/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:1251: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  color_vector = pd.Categorical(values.map(color_map))
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:1251: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  color_vector = pd.Categorical(values.map(color_map))


In [104]:
#Batch avware gene selection
sc.pp.highly_variable_genes(
    adata, n_top_genes=4000, flavor="cell_ranger", batch_key=batch_key
)
adata.var
n_batches = adata.var["highly_variable_nbatches"].value_counts()
ax = n_batches.plot(kind="bar")
n_batches
adata_hvg = adata[:, adata.var["highly_variable"]].copy()


/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:252: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:252: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:252: FutureWarning: The default o

In [105]:
neighbors_within_batch = 25 if adata_hvg.n_obs > 100000 else 3
neighbors_within_batch
adata_bbknn = adata_hvg.copy()
adata_bbknn.X = adata_bbknn.layers["sqrtcounts"].copy()
sc.pp.pca(adata_bbknn)


In [106]:
sce.pp.bbknn(
    adata_bbknn, batch_key=batch_key, neighbors_within_batch=neighbors_within_batch
)
adata_bbknn


AnnData object with n_obs × n_vars = 28334 × 4000
    obs: 'Orig.ident', 'Celltype', 'Dataset', 'Tissue', 'Organ', 'Condition', 'Genotype', 'Libraries', 'ACE', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_pt', 'log1p_total_counts_pt', 'pct_counts_pt', 'doublet', 'doublet_score', 'batch', 'n_counts'
    var: 'features', 'mt', 'pt', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'n_cells_by_counts-3', 'mean_cou

In [107]:
sc.tl.umap(adata_bbknn)
sc.pl.umap(adata_bbknn, color=[label_key, batch_key], wspace=1, save="_batch_effect_bbknn.png")

/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:1251: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  color_vector = pd.Categorical(values.map(color_map))
/home/sah2p/ondemand/tools/miniconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:1251: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  color_vector = pd.Categorical(values.map(color_map))


In [108]:
adata_bbknn


AnnData object with n_obs × n_vars = 28334 × 4000
    obs: 'Orig.ident', 'Celltype', 'Dataset', 'Tissue', 'Organ', 'Condition', 'Genotype', 'Libraries', 'ACE', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_pt', 'log1p_total_counts_pt', 'pct_counts_pt', 'doublet', 'doublet_score', 'batch', 'n_counts'
    var: 'features', 'mt', 'pt', 'n_cells_by_counts-0', 'mean_counts-0', 'log1p_mean_counts-0', 'pct_dropout_by_counts-0', 'total_counts-0', 'log1p_total_counts-0', 'n_cells_by_counts-1', 'mean_counts-1', 'log1p_mean_counts-1', 'pct_dropout_by_counts-1', 'total_counts-1', 'log1p_total_counts-1', 'n_cells_by_counts-2', 'mean_counts-2', 'log1p_mean_counts-2', 'pct_dropout_by_counts-2', 'total_counts-2', 'log1p_total_counts-2', 'n_cells_by_counts-3', 'mean_cou

In [109]:
import anndata2ri
import logging
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro
rcb.logger.setLevel(logging.ERROR)
from rpy2.robjects import pandas2ri
pandas2ri.activate()
anndata2ri.activate()
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


/tmp/ipykernel_2425043/4112179182.py:8: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [110]:
# sc.pp.normalize_per_cell(adata)
# sc.pp.sqrt(adata)
sce.pp.magic(adata_bbknn, name_list='all_genes', knn=5)


In [111]:
adata_bbknn.write(context_path+dataset+"_bbknn.h5ad")

# Appendix

In [ ]:
import re
import pandas as pd

# Extract gene names
gene_names = list(adata_obj.var_names)

# Define patterns for common naming conventions
patterns = {
    "Ensembl": re.compile(r"^ENS\w+"),
    "ZmLocus": re.compile(r"^Zm\d+[a-zA-Z]\d+$"),
    "MicroRNA": re.compile(r"^zma-MIR\d+[a-z]*$"),
    "Mitochondrial": re.compile(r"^(nad|cox|atp|rps|rrn|cob)\d*$"),
    "GeneSymbol": re.compile(r"^[A-Z0-9]+$")  # Common gene symbols
}

# Function to classify gene names
def classify_gene(gene_name):
    for category, pattern in patterns.items():
        if pattern.match(gene_name):
            return category
    return "Unknown"

# Apply classification
gene_categories = {gene: classify_gene(gene) for gene in gene_names}

# Convert to DataFrame for visualization
df = pd.DataFrame(list(gene_categories.items()), columns=["Gene", "Category"])

# Show unique categories
print(df["Category"].value_counts())

# Display some categorized genes
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Gene Naming Categories", dataframe=df)
df[df['Category'] == 'ZmLocus'].head(10)

In [79]:
import requests
import pandas as pd

def get_zea_mays_gene_ids():
    """
    Retrieve Zea mays gene annotations from Ensembl.
    Returns a dictionary with different gene ID types.
    """
    url = "https://rest.ensembl.org/lookup/symbol/zea_mays/"
    headers = {"Content-Type": "application/json"}
    
    # Gene ID types we are interested in
    gene_types = ["ensembl_gene_id", "entrezgene_id", "external_gene_name", "refseq_mrna"]

    # BioMart query for Zea mays genes
    biomart_url = "http://plants.ensembl.org/biomart/martservice"
    params = {
        "query": f"""
        <Query virtualSchemaName="plants_mart" formatter="TSV" header="1" uniqueRows="1">
            <Dataset name="zmays_eg_gene" interface="default">
                {"".join([f'<Attribute name="{gt}"/>' for gt in gene_types])}
            </Dataset>
        </Query>
        """
    }
    
    # Request BioMart data
    response = requests.get(biomart_url, params=params)
    if response.status_code != 200:
        raise ValueError("Failed to fetch gene annotations from Ensembl BioMart.")

    # Convert TSV data into DataFrame
    lines = response.text.strip().split("\n")
    df = pd.DataFrame([line.split("\t") for line in lines[1:]], columns=lines[0].split("\t"))
    print(df.head())
    return {col: set(df[col].dropna()) for col in df.columns}

def detect_gene_id_type(gene_list):
    """
    Detects the gene ID type of a given list of genes by comparing them with known Zea mays annotations.
    
    Parameters:
    - gene_list (list): List of gene identifiers from the dataset.
    
    Returns:
    - str: The detected gene ID type (e.g., "ENSEMBL", "ENTREZID", "SYMBOL", or "UNKNOWN").
    """
    # Get known Zea mays gene IDs
    gene_db = get_zea_mays_gene_ids()

    # Compare input gene list with each ID type
    best_match = None
    max_matches = 0

    for gene_type, known_ids in gene_db.items():
        matches = len(set(gene_list) & known_ids)
        if matches > max_matches:
            max_matches = matches
            best_match = gene_type

    return best_match if best_match else "UNKNOWN"

# Example Usage
gene_list = ["Zm00001eb393960", "Zm00001d000002"]  # Mix of Ensembl, RefSeq, Entrez
detected_type = detect_gene_id_type(gene_list)
print(f"Detected Gene ID Type: {detected_type}")

    Gene stable ID NCBI gene (formerly Entrezgene) ID Gene name RefSeq mRNA ID
0  Zm00001eb442760                                                            
1  Zm00001eb393960                          111590063                         
2  Zm00001eb393960                          111590227                         
3  Zm00001eb393960                          111590628                         
4  Zm00001eb393960                          111590787                         
Detected Gene ID Type: Gene stable ID
